<a href="https://colab.research.google.com/github/ayalarodrigues/LLL-RAG-Chatbot-Suporte/blob/main/LLM_RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação das bibliotecas


In [5]:
!pip install langchain langchain-groq langchain_community langchain-huggingface --q
!pip install faiss-cpu sentence-transformers PyMuPDF --q

#langchain-grop: acesso aos modelos via API
#faiss-cpu: onde serão armazenados os documentos(pode ser cpu ou gpu)
#sentence-transformers: conversão dos textos para números
#PyMuPDF: leitura e processamento de texto


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

Importações

In [6]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
import getpass

Carregamento da LLM

In [7]:
os.environ["GROQ_API_KEY"] = getpass.getpass()
#getpass não exibe a chave da API no CF


··········


In [8]:
def load_llm(id_model, temperature):
  llm = ChatGroq(
      model = id_model,
      temperature = temperature,
      max_tokens = None,
      timeout = None,
      max_retries = 2
  )
  return llm

In [9]:
#teste
id_model = "llama3-70b-8192" # @param {type: "string"}
temperature = 0.7 # @param {type: "slider", min: 0.1, max: 1.5, step: 0.1}

llm = load_llm(id_model, temperature)

In [10]:
#teste
prompt = "Como alterar minha senha?" # @param {type: "string"}

template = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente virtual prestativo e está respondendo perguntas gerais"),
    ("human", "{prompt}")
])

chain = template | llm | StrOutputParser()

res = chain.invoke({"prompt": prompt})
res

'Alterar a senha é uma boa prática de segurança!\n\nPara alterar sua senha, você precisará seguir os seguintes passos:\n\n**Se você estiver usando um site ou aplicativo:**\n\n1. Faça login em sua conta com sua senha atual.\n2. Clique em "Conta" ou "Perfil" e, em seguida, em "Segurança" ou "Senha".\n3. Clique em "Alterar senha" ou "Redefinir senha".\n4. Digite sua senha atual para confirmar sua identidade.\n5. Insira sua nova senha duas vezes (para confirmar).\n6. Clique em "Salvar" ou "Alterar senha" para confirmar as alterações.\n\n**Se você estiver usando um dispositivo móvel:**\n\n1. Abra o aplicativo ou vá para a tela de login.\n2. Toque em "Esqueceu a senha" ou "Redefinir senha".\n3. Insira seu endereço de e-mail ou nome de usuário associado à sua conta.\n4. Siga as instruções para redefinir sua senha (geralmente, você receberá um e-mail com um link para redefinir a senha).\n5. Insira sua nova senha duas vezes (para confirmar).\n6. Toque em "Salvar" ou "Alterar senha" para confirm

In [11]:
def show_res(res):
  from IPython.display import display, Markdown
  if "</think>" in res:
    res = res.split("</think>")[-1].strip()  #o modelo pode retornar o passo a passo necessário para resolução do problema
  else:
    res = res.strip()
  display(Markdown(res))


show_res(res)

 #a tag </think> serve para delimitar pensamentos intermediários, o que permite visualizar o "raciocínio" utilizado na resposta
 #optei por remover a tag por não ser de interesse da persona

Alterar a senha é uma boa prática de segurança!

Para alterar sua senha, você precisará seguir os seguintes passos:

**Se você estiver usando um site ou aplicativo:**

1. Faça login em sua conta com sua senha atual.
2. Clique em "Conta" ou "Perfil" e, em seguida, em "Segurança" ou "Senha".
3. Clique em "Alterar senha" ou "Redefinir senha".
4. Digite sua senha atual para confirmar sua identidade.
5. Insira sua nova senha duas vezes (para confirmar).
6. Clique em "Salvar" ou "Alterar senha" para confirmar as alterações.

**Se você estiver usando um dispositivo móvel:**

1. Abra o aplicativo ou vá para a tela de login.
2. Toque em "Esqueceu a senha" ou "Redefinir senha".
3. Insira seu endereço de e-mail ou nome de usuário associado à sua conta.
4. Siga as instruções para redefinir sua senha (geralmente, você receberá um e-mail com um link para redefinir a senha).
5. Insira sua nova senha duas vezes (para confirmar).
6. Toque em "Salvar" ou "Alterar senha" para confirmar as alterações.

**Dicas de segurança:**

* Use senhas fortes e únicas para cada conta.
* Evite usar informações pessoais, como seu nome ou data de nascimento, como senha.
* Altere suas senhas regularmente (a cada 60 a 90 dias) para manter sua segurança.
* Use um gerenciador de senhas seguro para armazenar e gerenciar suas senhas.

Lembre-se de que, se você tiver esquecido sua senha, você pode sempre redefini-la seguindo os passos acima. Se tiver alguma dúvida adicional, sinta-se à vontade para perguntar!

Definição do contexto

Há basicamente duas possibilidades para o exemplo de teste: o algoritmo não consegue retornar com exatidão o caminho exato para recuperar a senha ou a informação foi "inventada"(processo de alucinação).

O modelo quer apenas prever a próxima combinação de palavras que faça sentido. Nem sempre poderá ser entendido que o modelo não sabe essa informação com exatidão. O modelo pode apenas sugerir um caminho comum, como o retornado no exemplo, porém, o algoritmo não possui meios de conhecer o caminho exato sem consultar uma fonte que comprove.

Nesse aspecto, é importante que seja fornecido um contexto que tenha as informações exatas para a LLM e assim a dúvida do usuário pode ser respondida.

Para a obtenção do contexto, nesse estudo de caso, será feito o carregamento do documento que posteriormente será processado pelo pipeline de RAG.



In [12]:
#teste

context = """
Para alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.
Para alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.
"""

prompt = f"""
Como alterar minha senha?

Contexto: {context}
"""

In [13]:
#teste

prompt

"\nComo alterar minha senha?\n\nContexto: \nPara alterar uma senha no aplicativo, clique no menu 'Minha conta' e selecione 'Alterar senha'.\nPara alterar a senha pelo site, acesse 'Configurações' no menu do topo. Em seguida, selecione 'Minha conta' e 'Alterar senha'.\n\n"

In [14]:
#teste

res = chain.invoke({"prompt": prompt})
show_res(res)


Para alterar sua senha, você pode seguir os seguintes passos:

**No aplicativo:**

1. Clique no menu "Minha conta"
2. Selecione "Alterar senha"

**No site:**

1. Acesse o menu do topo e clique em "Configurações"
2. Selecione "Minha conta"
3. Clique em "Alterar senha"

Em seguida, você poderá inserir sua senha atual e criar uma nova senha. Lembre-se de escolher uma senha forte e única para proteger sua conta!